In [1]:
from quasinet.qnet import qdistance, save_qnet
from cognet.cognet import cognet as cg
from cognet.dataFormatter import dataFormatter
from cognet.model import model 
#import cognet.util
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
plt.style.use('fivethirtyeight')

In [2]:
## read in data, configure to Qnet specifications and fit Qnet model
data_obj=dataFormatter(samples="data/PTSD_cognet_test_processed.csv")
features,samples = data_obj.Qnet_formatter()

In [3]:
fit = False
model_obj = model()
if fit:
    model_obj.fit(data_obj=data_obj, njobs=2)
    # model_obj.save("examples_results/PTSD_cognet_test.joblib")
    save_qnet(model_obj.myQnet, "results/PTSD_cognet_test.joblib", low_mem=False)
else:
    model_obj.load("results/PTSD_cognet_test.joblib")

KeyboardInterrupt: 

In [ ]:
# load Qnet, write mpi files for distance matrix
# run qdistance matrix with "./mpi_run.sh" command
cognet_obj = cg()
cognet_obj.load_from_model(model_obj, data_obj, 'all')
cognet_obj.samples

# Compute dissonance and random mask reconstruction for Qnet samples

In [ ]:
# compute dissonance matrix of qnet samples
cognet_obj.set_nsamples(304)
cognet_obj.MAX_PROCESSES = 11
return_dict = cognet_obj.dissonance_matrix(outfile="mpi_tmp/PTSD_dissonance_matrix.csv", processes=2)
return_dict
qnet_dissonance_df=pd.DataFrame(return_dict.copy())

# computing random mask reconstruction of qnet samples
print("___________________________________________________")
cognet_obj.set_nsamples(304)
cognet_obj.steps = 1000
qnet_randommask_df = cognet_obj.randomMaskReconstruction_multiple('results/PTSD_randomMaskRecon_test.csv',  save_samples=True)
qnet_randommask_df

In [ ]:
qnet_dissonance_df.mean(axis=1).hist()
plt.title("Dissonance of Qnet Samples")
plt.xlabel("Dissonance")
plt.ylabel("Frequency")
plt.show()

In [ ]:
cognet_obj.steps = 200
# computing random mask reconstruction of qnet samples
print("___________________________________________________")
cognet_obj.set_nsamples(304)
qnet_randommask_df_200step = cognet_obj.randomMaskReconstruction_multiple('results/PTSD_randomMaskRecon_test.csv',  save_samples=False)
qnet_randommask_df_200step

In [ ]:
# redefine the reconstruction error for PTSD
samples=[]
for s in qnet_randommask_df['sample']:
    samples.append(list(s))
qnet_randommask_samples=pd.DataFrame(data=samples, columns=features, dtype='int').astype(int)
qnet_randommask_samples

qsamples=[]
for s in qnet_randommask_df['qsampled']:
    qsamples.append(list(s))
qnet_randommask_qsamples=pd.DataFrame(data=qsamples, columns=features, dtype='int').replace('',0).astype(int)

In [ ]:
# calculate the recon error
diff_df = qnet_randommask_samples - qnet_randommask_qsamples
diff_df1 = diff_df.copy()
diff_df["diff sum"] = diff_df.sum(axis=1)
num_masked = pd.DataFrame([len(list(s)) for s in qnet_randommask_df['mask_']], columns=["num masked"])
diff_df["num masked"] = num_masked
diff_df["recon_results"] = diff_df["diff sum"] / diff_df["num masked"]

# add PTSD indicator
PTSD_indicator = pd.read_csv("data/PTSD_cognet_test.csv")[["PTSDDx"]]

print(diff_df1[PTSD_indicator["PTSDDx"] == 0].mean().mean(),diff_df1[PTSD_indicator["PTSDDx"] == 1].mean().mean())

In [ ]:
diff_df1[PTSD_indicator["PTSDDx"] == 0]

In [ ]:
diff_df1[PTSD_indicator["PTSDDx"] == 1]

In [ ]:
len(diff_df1[PTSD_indicator["PTSDDx"] == 1])

In [ ]:
# plot the recon error grouped by PTSD indicator
plt.figure(figsize=[20,6])
plt.subplot(2,1,1)
plt.subplots_adjust(hspace=.5)
diff_df1[PTSD_indicator["PTSDDx"] == 0].mean().plot(kind='bar',color='r')
ax0=plt.gca()

plt.subplot(2,1,2)
diff_df1[PTSD_indicator["PTSDDx"] == 1].mean().plot(kind='bar',color='k')
ax=plt.gca()

[label.set_visible(False) for x in [ax, ax0] for label in x.xaxis.get_ticklabels()[::2]]
ax0.set_title('mean reconstruction error',fontsize=20,fontweight='bold')
ax0.legend(['positive'],fontsize=18)
ax.legend(['negative'],fontsize=18)

# plt.gca().set_xlabel('')
# diff_df1.mean().plot(kind="bar")
# ax=plt.gca()
# [label.set_visible(False) for label in ax.xaxis.get_ticklabels()[::2]]
# ax.set_title('mean reconstruction error',fontsize=15,fontweight='bold')

# Drawing Random Samples from Uniform Distribution

In [ ]:
print("__________________________________________________")
n = 304
print("{} random samples computed".format(n))
usamples = cognet_obj.random_sample(type="uniform",n=n, n_jobs=3)
results = []
for s in range(len(usamples)):
    results.append(cognet_obj.dissonance(0, sample=usamples.iloc[s]))
urandom_dissonance_df = pd.DataFrame(results)
urandom_dissonance_df

In [ ]:
# find confidence interval for random samples drawn from uniform distributions
qnet_mean = urandom_dissonance_df.mean(axis=1).mean()
qnet_std = urandom_dissonance_df.mean(axis=1).std(ddof=1)
alpha_p1 = 0.1
alpha_p05 = 0.05
n_sided = 1 # 1-sided test
z_crit = stats.norm.ppf(1-alpha_p1/n_sided)
threshold_p1=(-z_crit*qnet_std)+qnet_mean

z_crit = stats.norm.ppf(1-alpha_p05/n_sided)
threshold_p05=(-z_crit*qnet_std)+qnet_mean

print('Uniform Random Sampling Threshold (90%): ',threshold_p1)
print('Uniform Random Sampling Threshold (95%): ',threshold_p05)

plt.figure()
udissonance_df = pd.DataFrame(data=qnet_dissonance_df.mean(axis=1), columns=["Qnet"])
udissonance_df["random"] = urandom_dissonance_df.mean(axis=1)
plt.hist(udissonance_df["Qnet"], alpha=0.5, label="Qnet samples")
plt.hist(udissonance_df["random"], alpha=0.5, label="random samples")
plt.legend()
plt.axvline(threshold_p05, color="red", linestyle="--", alpha=.8)
plt.show()

# Comparing Positive vs Negative PTSD dissonance and reconstruction results

In [ ]:
# getting positive and negative PTSD samples and merging with dissonance and reconstruction results
PTSD_DATA = pd.read_csv("data/PTSD_cognet_test.csv")[["PTSDDx"]]

PTSD_DATA["Mean Reconstruction"] = diff_df["recon_results"] # qnet_randommask_df["rederr"]
PTSD_DATA["Mean Dissonance"] = qnet_dissonance_df.mean(axis=1)

plt.style.use('fivethirtyeight')
PTSD_DATA.plot.scatter("Mean Dissonance", "Mean Reconstruction", c="PTSDDx")

In [ ]:
# plotting negative vs postive PTSD samples
#random_mask_dissonance_df.plot.scatter("Mean Dissonance", "Mean Reconstruction", c="actual")
plt.axvline(threshold_p05, color="red", linestyle="--", alpha=.8)
sns.scatterplot(PTSD_DATA["Mean Dissonance"],
                PTSD_DATA["Mean Reconstruction"],
                hue=PTSD_DATA["PTSDDx"]).set(ylim=(-3, 3))

In [ ]:
print("Number of negative PTSD results that are 'suspect' at alpha level of .05:", len(PTSD_DATA[(PTSD_DATA["Mean Dissonance"] >= threshold_p05) & (PTSD_DATA["PTSDDx"] == 0)]["Mean Dissonance"]))
print("Number of positive PTSD results that are 'suspect' at alpha level of .05:", len(PTSD_DATA[(PTSD_DATA["Mean Dissonance"] >= threshold_p05) & (PTSD_DATA["PTSDDx"] == 1)]["Mean Dissonance"]))

In [ ]:
PTSD_DATA[(PTSD_DATA["Mean Dissonance"] >= threshold_p05)]